## Apress - Industrialized Machine Learning Examples

Andreas Francois Vermeulen
2019

### This is an example add-on to a book and needs to be accepted as part of that copyright.

# Chapter-014-11-Space-Fuel-Sim-01

![RIF plus Data Lake](../../images/spaceship.png)

In [1]:
import random
import simpy


RANDOM_SEED = 1968
SIM_TIME = 120

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
def HighwayA(name, env, out_pipe):
    """A process which randomly generates clients."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(6, 10))

        msg = (env.now, '%s client from Space Highway A needs fuel at %d' % (name, env.now))
        out_pipe.put(msg)

In [4]:
def HighwayB(name, env, out_pipe):
    """A process which randomly generates clients."""
    while True:
        # wait for next transmission
        yield env.timeout(random.randint(1, 4))

        msg = (env.now, '%s client from Space Highway B needs fuel at %d' % (name, env.now))
        out_pipe.put(msg)

In [5]:
class FuelLine(object):
    def __init__(self, env, capacity=simpy.core.Infinity):
        self.env = env
        self.capacity = capacity
        self.pipes = []

    def put(self, value):
        """Transfer to all fuel pumps."""
        if not self.pipes:
            raise RuntimeError('There are no output pumps.')
        events = [store.put(value) for store in self.pipes]
        return self.env.all_of(events)  # Condition event for all "events"

    def get_output_conn(self):
        pipe = simpy.Store(self.env, capacity=self.capacity)
        self.pipes.append(pipe)
        return pipe

In [6]:
def FuelClient(name, env, in_pipe):
    """A process which consumes fuel."""
    while True:
        msg = yield in_pipe.get()

        if msg[0] < env.now:
            print('No fuel: at time %d: %s received fuel request: %s' %
                  (env.now, name, msg[1]))

        else:
            print('at time %d: %s received fuel request: %s.' %
                  (env.now, name, msg[1]))

        # Process is interfering with your work!!!
        # The new fuel connections is stiff.
        
        yield env.timeout(random.randint(1, 2))

In [7]:
# Setup and start the simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()

FuelLineA = FuelLine(env)
FuelLineB = FuelLine(env)

env.process(HighwayA('Fuel-A', env, FuelLineA))
env.process(HighwayB('Fuel-B', env, FuelLineB))

env.process(FuelClient('Spaceship A', env, FuelLineA.get_output_conn()))
env.process(FuelClient('Spaceship B', env, FuelLineB.get_output_conn()))
env.process(FuelClient('Spaceship C', env, FuelLineB.get_output_conn()))

print('\nFuel Station Simulation\n')
env.run(until=SIM_TIME)


Fuel Station Simulation

at time 1: Spaceship B received fuel request: Fuel-B client from Space Highway B needs fuel at 1.
at time 1: Spaceship C received fuel request: Fuel-B client from Space Highway B needs fuel at 1.
at time 4: Spaceship B received fuel request: Fuel-B client from Space Highway B needs fuel at 4.
at time 4: Spaceship C received fuel request: Fuel-B client from Space Highway B needs fuel at 4.
at time 8: Spaceship B received fuel request: Fuel-B client from Space Highway B needs fuel at 8.
at time 8: Spaceship C received fuel request: Fuel-B client from Space Highway B needs fuel at 8.
at time 9: Spaceship C received fuel request: Fuel-B client from Space Highway B needs fuel at 9.
at time 9: Spaceship A received fuel request: Fuel-A client from Space Highway A needs fuel at 9.
No fuel: at time 10: Spaceship B received fuel request: Fuel-B client from Space Highway B needs fuel at 9
at time 10: Spaceship C received fuel request: Fuel-B client from Space Highway B n

## Done

In [8]:
import datetime
now = datetime.datetime.now()
print('Done!',str(now))

('Done!', '2019-05-03 15:40:14.745000')
